# Import Required Libraries
Import the necessary libraries, including LeagueHistoryLoader.

In [1]:
# LeagueHistoryLoader for loading the league history data
from src.fpl_load import LeagueHistoryLoader
from src.questions import get_points_by_gameweek

# Setting the league id
league_id = 665568

# Creating an instance of LeagueHistoryLoader
history_loader = LeagueHistoryLoader(league_id)

In [2]:
# Loading the league history data
league_data = history_loader.get_data()

df = league_data
import duckdb
import pandas as pd

# increase number of rows displayable in pandas
pd.set_option('display.max_rows', 500)


# Convert DataFrame to DuckDB
duckdb_df = duckdb.from_df(df)

In [26]:
df

,event,event_points,cumulative_points,fpl_event_rank,fpl_event_rank_sort,overall_rank,percentile_rank,bank,team_value,event_transfers,event_transfers_cost,points_on_bench,entry,id,event_total,player_name,league_rank,last_rank,rank_sort,standings_total,entry_name
0,1,80,80,710070,784384,710070,NaN,0,1000,0,0,11,3110982,21377033,67,Dominic Rech,1,1,1,2532,In Good Kompany
1,2,52,132,2319915,2363147,812849,NaN,10,1002,1,0,21,3110982,21377033,67,Dominic Rech,1,1,1,2532,In Good Kompany
2,3,53,185,1862688,1894075,531104,NaN,6,1008,1,0,20,3110982,21377033,67,Dominic Rech,1,1,1,2532,In Good Kompany
3,4,64,249,7650408,7666623,1455389,NaN,19,1013,1,0,8,3110982,21377033,67,Dominic Rech,1,1,1,2532,In Good Kompany
4,5,38,287,7572009,7616547,2126814,NaN,17,1011,1,0,7,3110982,21377033,67,Dominic Rech,1,1,1,2532,In Good Kompany
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
603,34,89,1873,3298387,3315662,2732108,35.0,0,1014,2,4,0,5025791,39069622,89,Fin Whittington,16,16,16,2191,Is Your Motherwell
604,35,71,1944,1321532,1340433,2508957,15.0,8,1015,1,0,0,5025791,39069622,89,Fin Whittington,16,16,16,2191,Is Your Motherwell
605,36,74,2018,4891535,4965522,2508761,50.0,8,1016,0,0,6,5025791,39069622,89,Fin Whittington,16,16,16,2191,Is Your Motherwell
606,37,84,2102,4844371,4887328,2683107,45.0,8,1015,0,0,8,5025791,39069622,89,Fin Whittington,16,16,16,2191,Is Your Motherwell


In [3]:
game_week_points = get_points_by_gameweek(df)
game_week_points.head(20)

,player_name,entry_name,gameweek,points
0,Matt Harmer,Worrall Loada C*ap,1,66.0
1,Prince Ehigiator,McCris-P FC,1,80.0
2,Fin Whittington,Is Your Motherwell,1,76.0
3,Patrick Prendergast,Who Got The Assist,1,76.0
4,Andrew Wallace,Boggie FC,1,74.0
5,Jack Grimley,Cancelo Culture,1,82.0
6,Mark Longmire,Botman Begins,1,77.0
7,Dominic Rech,In Good Kompany,1,80.0
8,Rory McCullough,AthletEcho Falls FC,1,63.0
9,William Levick,Haalandaise Sauce,1,60.0


In [4]:
import pandas as pd
import numpy as np
import locale
import plotly.express as px
import plotly.graph_objects as go
import streamlit as st
import streamlit.components.v1 as components
import io
import bar_chart_race as bcr
import base64

from IPython.display import Video

@st.cache_data
def plot_graph_race(df_data):
    df_values, df_ranks = bcr.prepare_long_data(df_data, 
                index='gameweek', 
                columns='entry_name', 
                values='points', 
                steps_per_period=1)

    return bcr.bar_chart_race(df_values,
                n_bars=16, 
                steps_per_period=30, 
                period_length=1500, 
                title = 'League Race', 
                period_template='{x:.0f}', 
                fixed_max=True, 
                filter_column_colors=True).data

html_str = plot_graph_race(game_week_points)

start = html_str.find('base64,')+len('base64,')
end = html_str.find('">')

video = base64.b64decode(html_str[start:end])
st.video(video)

/Users/echandler/Documents/projects/fpl_blogs/fpl_league_wrapped/venv/lib/python3.10/site-packages/bar_chart_race/_utils.py:204: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_wide = df.pivot_table(index=index, columns=columns, values=values,
/Users/echandler/Documents/projects/fpl_blogs/fpl_league_wrapped/venv/lib/python3.10/site-packages/bar_chart_race/_utils.py:111: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_values.iloc[:, 0] = df_values.iloc[:, 0].fillna(method='ffill')
/Users/echandler/Documents/projects/fpl_blogs/fpl_league_wrapped/venv/lib/python3.10/site-packages/bar_chart_race/_utils.py:111: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_values.iloc[:, 0] = df_values.iloc[:, 0].fillna(method='ffill')


Exception: Requested MovieWriter (ffmpeg) not available

In [ ]:
Video(anim, embed=True)

NameError: name 'anim' is not defined

# Answer Initial Questions
Use the loaded data to answer the questions found in the Initial Questions Answered section in README.md.

In [13]:
from IPython.display import display
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# From the get_rank result, create a function that calculates the highest, lowest and average rank for each player using SQL
def get_player_rank(duckdb_df):
    """
    This function returns a DataFrame with the highest, lowest and average rank for each player
    """
    return duckdb.query("""
        WITH total_points AS (
            SELECT 
                event, 
                event_points, 
                entry_name, 
                player_name,
                SUM(event_points) OVER (PARTITION BY player_name, entry_name ORDER BY event) AS total_points
            FROM
                duckdb_df
        )
        SELECT 
            player_name,
            entry_name,
            MAX(rank) as worst_rank,
            MIN(rank) as best_rank,
            AVG(rank) as average_rank
        FROM (
            SELECT 
                event, 
                event_points, 
                entry_name, 
                player_name,
                total_points,
                RANK() OVER (PARTITION BY event ORDER BY total_points DESC) as rank
            FROM
                total_points
        )
        GROUP BY
            player_name,
            entry_name
    """).to_df()

In [22]:
def get_player_worst_rank_event(duckdb_df):
    """
    This function returns a DataFrame with the player_name, entry_name, worst_rank, and the event on which that worst_rank happened
    """
    return duckdb.query("""
        WITH total_points AS (
            SELECT 
                event, 
                event_points, 
                entry_name, 
                player_name,
                SUM(event_points) OVER (PARTITION BY player_name, entry_name ORDER BY event) AS total_points
            FROM
                duckdb_df
        ),
        ranks AS (
            SELECT 
                event, 
                event_points, 
                entry_name, 
                player_name,
                total_points,
                RANK() OVER (PARTITION BY event ORDER BY total_points DESC) as rank
            FROM
                total_points
        ),
        worst_ranks AS (
            SELECT 
                player_name,
                entry_name,
                MAX(rank) as worst_rank
            FROM 
                ranks
            GROUP BY
                player_name,
                entry_name
        )
        SELECT 
            wr.player_name,
            wr.entry_name,
            wr.worst_rank,
            STRING_AGG(CAST(r.event AS VARCHAR), ', ' ORDER BY event) AS event_list
        FROM
            worst_ranks wr
        JOIN 
            ranks r
        ON 
            wr.player_name = r.player_name AND 
            wr.entry_name = r.entry_name AND 
            wr.worst_rank = r.rank
        GROUP BY
            wr.player_name,
            wr.entry_name,
            wr.worst_rank
        ORDER BY 
            wr.player_name
    """).to_df()

get_player_worst_rank_event(duckdb_df)

,player_name,entry_name,worst_rank,event_list
0,Alex Kitson,4-4-*******-2,16,"3, 10, 11, 12, 13, 14, 15, 16, 17, 18"
1,Andrew Wallace,Boggie FC,10,31
2,Daniel Brown,Benson and Hedges,16,7
3,Dominic Rech,In Good Kompany,10,"4, 5"
4,Fin Whittington,Is Your Motherwell,16,"23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34..."
5,Jack Chalkley,Garnachos,10,1
6,Jack Grimley,Cancelo Culture,15,"27, 28, 30, 36"
7,Liam Aylwin,Unbelievable Jeff!!,11,34
8,Mark Longmire,Botman Begins,10,19
9,Mason Scott,FC Mase,10,"2, 23, 29, 30"


In [23]:
def get_player_best_rank_event(duckdb_df):
    """
    This function returns a DataFrame with the player_name, entry_name, best_rank, and the event on which that best_rank happened
    """
    return duckdb.query("""
        WITH total_points AS (
            SELECT 
                event, 
                event_points, 
                entry_name, 
                player_name,
                SUM(event_points) OVER (PARTITION BY player_name, entry_name ORDER BY event) AS total_points
            FROM
                duckdb_df
        ),
        ranks AS (
            SELECT 
                event, 
                event_points, 
                entry_name, 
                player_name,
                total_points,
                RANK() OVER (PARTITION BY event ORDER BY total_points DESC) as rank
            FROM
                total_points
        ),
        best_ranks AS (
            SELECT 
                player_name,
                entry_name,
                MIN(rank) as best_rank
            FROM 
                ranks
            GROUP BY
                player_name,
                entry_name
        )
        SELECT 
            br.player_name,
            br.entry_name,
            br.best_rank,
            STRING_AGG(CAST(r.event AS VARCHAR), ', ' ORDER BY event) AS event_list
        FROM
            best_ranks br
        JOIN 
            ranks r
        ON 
            br.player_name = r.player_name AND 
            br.entry_name = r.entry_name AND 
            br.best_rank = r.rank
        GROUP BY
            br.player_name,
            br.entry_name,
            br.best_rank
        ORDER BY 
            br.player_name
    """).to_df()

get_player_best_rank_event(duckdb_df)

,player_name,entry_name,best_rank,event_list
0,Alex Kitson,4-4-*******-2,5,"29, 30"
1,Andrew Wallace,Boggie FC,2,"2, 14, 16, 17, 18, 21, 22, 23"
2,Daniel Brown,Benson and Hedges,2,5
3,Dominic Rech,In Good Kompany,1,"8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, ..."
4,Fin Whittington,Is Your Motherwell,5,4
5,Jack Chalkley,Garnachos,3,"5, 6, 19, 20, 21"
6,Jack Grimley,Cancelo Culture,1,2
7,Liam Aylwin,Unbelievable Jeff!!,1,3
8,Mark Longmire,Botman Begins,2,"10, 11, 12, 13, 24, 25, 26, 30, 32, 33"
9,Mason Scott,FC Mase,1,"1, 4, 5, 6, 7"


In [14]:
# Work out the highest and lowest postition for each player throughout the season
player_rank = get_player_rank(duckdb_df)
player_rank[['player_name', 'entry_name', 'worst_rank']]

,player_name,entry_name,worst_rank
0,Alex Kitson,4-4-*******-2,16
1,Matt Harmer,Worrall Loada C*ap,15
2,William Levick,Haalandaise Sauce,16
3,Jack Grimley,Cancelo Culture,15
4,Andrew Wallace,Boggie FC,10
5,Dominic Rech,In Good Kompany,10
6,Mark Longmire,Botman Begins,10
7,Rory McCullough,AthletEcho Falls FC,15
8,Mason Scott,FC Mase,10
9,Patrick Prendergast,Who Got The Assist,16


In [ ]:
# Work out the highest and lowest postition for each player throughout the season
player_rank = get_player_rank(duckdb_df)
player_rank[['player_name', 'entry_name', 'best_rank']]

,player_name,entry_name,worst_rank
0,Alex Kitson,4-4-*******-2,16
1,Matt Harmer,Worrall Loada C*ap,15
2,William Levick,Haalandaise Sauce,16
3,Jack Grimley,Cancelo Culture,15
4,Andrew Wallace,Boggie FC,10
5,Dominic Rech,In Good Kompany,10
6,Mark Longmire,Botman Begins,10
7,Rory McCullough,AthletEcho Falls FC,15
8,Mason Scott,FC Mase,10
9,Patrick Prendergast,Who Got The Assist,16


In [6]:
import plotly.express as px

df = get_player_rank(duckdb_df)

# Create 3 bar charts for the highest, lowest and average rank for each entry_name
# Order the bars by the highest rank

fig_highest = px.bar(df, x='player_name', y='highest_rank', title='Highest Rank')
fig_highest.show()

fig_lowest = px.bar(df, x='player_name', y='lowest_rank', title='Lowest Rank')
fig_lowest.show()

fig_average = px.bar(df, x='player_name', y='average_rank', title='Average Rank')
fig_average.show()

In [13]:
def get_total_points_and_bench_points(duckdb_df):
    """
    This function returns a DataFrame with the total points and total points left on the bench for each player and team for the season.
    """
    return duckdb.query(
        """
        SELECT 
            player_name, 
            entry_name, 
            SUM(points_on_bench) AS bench_points, 
            SUM(event_points) AS total_points
        FROM 
            duckdb_df
        GROUP BY 
            player_name, 
            entry_name
    """
    ).to_df()
get_total_points_and_bench_points(duckdb_df)

,player_name,entry_name,bench_points,total_points
0,Oliver Bignall,Biggie FC,262.0,2403.0
1,Jack Chalkley,Garnachos,290.0,2361.0
2,Mason Scott,FC Mase,223.0,2314.0
3,Prince Ehigiator,McCris-P FC,257.0,2260.0
4,Alex Kitson,4-4-*******-2,159.0,2295.0
5,Dominic Rech,In Good Kompany,290.0,2505.0
6,Andrew Wallace,Boggie FC,298.0,2342.0
7,Mark Longmire,Botman Begins,276.0,2332.0
8,Rory McCullough,AthletEcho Falls FC,170.0,2254.0
9,Jack Grimley,Cancelo Culture,292.0,2216.0


In [3]:


total_points_left_on_bench = duckdb.query("""
        SELECT 
            player_name, 
            COUNT(*) AS times_last_rank
        FROM (
            SELECT 
                event AS gameweek, 
                player_name
            FROM 
                duckdb_df
            WHERE 
                rank = (SELECT MAX(rank) AS FROM duckdb_df d2 WHERE d2.event = duckdb_df.event)
        ) subquery
        GROUP BY 
            player_name
        ORDER BY 
            times_last_rank DESC
        LIMIT 1
    """
    ).to_df()

BinderException: Binder Error: Referenced column "rank" not found in FROM clause!
Candidate bindings: "d2.bank", "d2.rank_sort", "d2.last_rank"

In [15]:
import duckdb

# Convert DataFrame to DuckDB
duckdb_df = duckdb.from_df(df)

# mydf = pd.DataFrame({'a' : [1, 2, 3]})

# print(duckdb.query("SELECT SUM(a) FROM mydf").to_df())

# Most points left on bench in a week
total_points_left_on_bench = duckdb.query("""
    SELECT player_name, entry_name, SUM(points_on_bench) AS bench_points
    FROM duckdb_df
    GROUP BY player_name, entry_name
    ORDER BY bench_points DESC
""").to_df()

most_points_left_on_bench_week = duckdb.query("""
    SELECT d.player_name, d.entry_name, d.points_on_bench AS most_points_left_on_bench, d.event
    FROM duckdb_df d
    JOIN (
        SELECT player_name, entry_name, MAX(points_on_bench) AS max_points
        FROM duckdb_df
        GROUP BY player_name, entry_name
    ) m ON d.player_name = m.player_name AND d.entry_name = m.entry_name AND d.points_on_bench = m.max_points
    ORDER BY most_points_left_on_bench DESC
""").to_df()

In [16]:
most_points_left_on_bench_week

,player_name,entry_name,most_points_left_on_bench,event
0,Daniel Brown,Benson and Hedges,28,6
1,Oliver Bignall,Biggie FC,27,18
2,Mark Longmire,Botman Begins,24,14
3,Patrick Prendergast,Who Got The Assist,24,23
4,Liam Aylwin,Unbelievable Jeff!!,22,21
5,Dominic Rech,In Good Kompany,21,2
6,Jack Chalkley,Garnachos,19,20
7,Mason Scott,FC Mase,19,5
8,Jack Grimley,Cancelo Culture,19,14
9,Prince Ehigiator,McCris-P FC,18,19


In [53]:
biggest_difference = duckdb.query("""
    SELECT a.event, a.player_name AS player1, a.entry_name AS entry1, a.event_points AS points1,
           b.player_name AS player2, b.entry_name AS entry2, b.event_points AS points2,
           ABS(a.event_points - b.event_points) AS difference
    FROM duckdb_df a, duckdb_df b
    WHERE a.event = b.event AND a.player_name != b.player_name
    ORDER BY difference DESC
    LIMIT 1
""").to_df()
biggest_difference

,event,player1,entry1,points1,player2,entry2,points2,difference
0,23,Jack Grimley,Cancelo Culture,44,William Levick,Haalandaise Sauce,110,66


In [6]:
# Most points left on bench in a week
most_points_left_on_bench = df['points_on_bench'].max()
print(f"Most points left on bench: {most_points_left_on_bench}")


Most points left on bench: 28


In [ ]:

# Worst week
worst_week = df['event_total_points'].idxmin()
print(f"Worst week: {worst_week}")


In [ ]:

# Biggest difference in points on a week from winner and loser
df_grouped = df.groupby('event')['event_total_points'].agg(['min', 'max'])
df_grouped['difference'] = df_grouped['max'] - df_grouped['min']
biggest_difference = df_grouped['difference'].max()
print(f"Biggest difference in points on a week from winner and loser: {biggest_difference}")